# N2 molecule eigenvalue estimation using vqe and qpe and subsequently ITPE

In [1]:
import numpy as np
import os
import sys
import pyscf
from qiskit import QuantumCircuit
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.circuit.library import PhaseEstimation, TwoLocal
from qiskit.circuit.library import HamiltonianGate
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator

### import nature library
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitMapper
import time
import inspect
from pprint import pprint

In [2]:
# Define the molecule
driver = PySCFDriver(
    atom="N 0 0 0; N 0 0 1.0975",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

# Run the driver to get the problem
problem = driver.run()

# Define the log output (here using stdout)
flog = sys.stdout

### Apply freezecore transformer
transformer = FreezeCoreTransformer()
problem = transformer.transform(problem)

hamiltonian = problem.hamiltonian.second_q_op()
#print(hamiltonian)
print("\n".join(str(hamiltonian).splitlines()[:10] + ["..."]))

Fermionic Operator
number spin orbitals=16, number terms=3288
  0.3741022545470474 * ( +_0 +_0 -_0 -_0 )
+ -0.05443765442389172 * ( +_0 +_0 -_4 -_0 )
+ 0.2560942010813374 * ( +_0 +_1 -_1 -_0 )
+ -0.03671071200315422 * ( +_0 +_1 -_7 -_0 )
+ 0.30888873007502443 * ( +_0 +_2 -_2 -_0 )
+ 0.3088887300750244 * ( +_0 +_3 -_3 -_0 )
+ -0.05443765442389172 * ( +_0 +_4 -_0 -_0 )
+ 0.27134250457552006 * ( +_0 +_4 -_4 -_0 )
...


In [3]:
print(problem.num_particles)
print(problem.num_spatial_orbitals)
#print(problem.active_orbitals)

(5, 5)
8


In [4]:
mapper = JordanWignerMapper()

pauli_hamiltonian = mapper.map(hamiltonian)

for pauli, coeff in sorted(pauli_hamiltonian.label_iter()):
    print(f"{coeff.real:+.8f} * {pauli}")

-25.66868383 * IIIIIIIIIIIIIIII
+1.06406699 * IIIIIIIIIIIIIIIZ
+0.68436479 * IIIIIIIIIIIIIIZI
+0.11401214 * IIIIIIIIIIIIIIZZ
+0.66055804 * IIIIIIIIIIIIIZII
+0.12815933 * IIIIIIIIIIIIIZIZ
+0.11484539 * IIIIIIIIIIIIIZZI
+0.66055804 * IIIIIIIIIIIIZIII
+0.12815933 * IIIIIIIIIIIIZIIZ
+0.11484539 * IIIIIIIIIIIIZIZI
+0.12909402 * IIIIIIIIIIIIZZII
+0.01521989 * IIIIIIIIIIIXIZZX
+0.01521989 * IIIIIIIIIIIXZIZX
+0.02295002 * IIIIIIIIIIIXZZIX
+0.03709120 * IIIIIIIIIIIXZZZX
+0.01521989 * IIIIIIIIIIIYIZZY
+0.01521989 * IIIIIIIIIIIYZIZY
+0.02295002 * IIIIIIIIIIIYZZIY
+0.03709120 * IIIIIIIIIIIYZZZY
+0.60517632 * IIIIIIIIIIIZIIII
+0.12279850 * IIIIIIIIIIIZIIIZ
+0.08338043 * IIIIIIIIIIIZIIZI
+0.12247107 * IIIIIIIIIIIZIZII
+0.12247107 * IIIIIIIIIIIZZIII
-0.00041831 * IIIIIIIIIIXXIXXI
+0.02954458 * IIIIIIIIIIXXIYYI
+0.00001878 * IIIIIIIIIIXXXZXI
-0.00132661 * IIIIIIIIIIXXYZYI
-0.02996289 * IIIIIIIIIIXYIYXI
+0.00134539 * IIIIIIIIIIXYYZXI
+0.00060002 * IIIIIIIIIIXZXIXX
+0.00024949 * IIIIIIIIIIXZXIYY
+0.0003

In [5]:
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

as_transformer = ActiveSpaceTransformer(2,2) # syntax = AST(num_electrons,num_orbitals)
as_problem = as_transformer.transform(problem)

print(as_problem.num_particles)
print(as_problem.num_spatial_orbitals)
print(as_problem.hamiltonian.electronic_integrals.alpha)

(1, 1)
2
Polynomial Tensor
 "+-":
array([[-1.12465081e+00, -5.20086815e-17],
       [-5.20086815e-17, -7.62075705e-01]])
 "++--":
array([[[[ 5.85155853e-01, -1.92106743e-17],
         [-1.92106743e-17,  5.40817488e-01]],

        [[-1.92106743e-17,  3.82404172e-02],
         [ 3.82404172e-02, -1.05089154e-17]]],


       [[[-1.92106743e-17,  3.82404172e-02],
         [ 3.82404172e-02, -1.05089154e-17]],

        [[ 5.40817488e-01, -1.05089154e-17],
         [-1.05089154e-17,  6.05101909e-01]]]])


In [6]:
hamiltonian = as_problem.hamiltonian.second_q_op()

#### Hilbert space reduction for pauli hamiltonian using AST

In [7]:
#convert second quantized operator into pauli strings using any chemistry encoding of choice. we use jw.
mapper = JordanWignerMapper()

pauli_hamiltonian = mapper.map(hamiltonian)

for pauli, coeff in sorted(pauli_hamiltonian.label_iter()):
    print(f"{coeff.real:+.8f} * {pauli}")

-1.06746480 * IIII
+0.15518780 * IIIZ
-0.03108626 * IIZI
+0.12564427 * IIZZ
+0.15518780 * IZII
+0.14628896 * IZIZ
+0.13520437 * IZZI
+0.00956010 * XXXX
+0.00956010 * XXYY
+0.00956010 * YYXX
+0.00956010 * YYYY
-0.03108626 * ZIII
+0.13520437 * ZIIZ
+0.15127548 * ZIZI
+0.12564427 * ZZII


In [8]:
pauli_hamiltonian

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IIZZ', 'IZII', 'IZIZ', 'ZIII', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-1.0674648 +0.j,  0.1551878 +0.j, -0.03108626+0.j,  0.12564427+0.j,
  0.1551878 +0.j,  0.14628896+0.j, -0.03108626+0.j,  0.13520437+0.j,
  0.0095601 +0.j,  0.0095601 +0.j,  0.0095601 +0.j,  0.0095601 +0.j,
  0.13520437+0.j,  0.15127548+0.j,  0.12564427+0.j])

In [9]:
eig = np.linalg.eigvals(pauli_hamiltonian)
print('NUMPY eig',np.min(eig))

estimator = Estimator()
optimizer = SLSQP()
ansatz = TwoLocal(rotation_blocks=['ry', 'rz'], entanglement_blocks='cz')

vqe = VQE(estimator, ansatz, optimizer)
result = vqe.compute_minimum_eigenvalue(operator=pauli_hamiltonian)
print('VQE eig', result.eigenvalue)

NUMPY eig (-1.666103229556738+0j)
VQE eig -1.6641452821940297


In [10]:
n_qpe_qbits = 10

U = HamiltonianGate(pauli_hamiltonian, 1, label='H')

# Obtain a solution via QPE
total_qbits = U.num_qubits + n_qpe_qbits
measure_circ = QuantumCircuit(total_qbits, n_qpe_qbits)
measure_circ.h([-1, -2])

qpe = PhaseEstimation(n_qpe_qbits, U)
Qpe=qpe.decompose().decompose().decompose().decompose().decompose().decompose().decompose()
measure_circ = measure_circ.compose(Qpe)
#measure_circ.draw("mpl")

measure_circ.measure(range(n_qpe_qbits), range(n_qpe_qbits))

In [11]:
from qiskit_aer import Aer
backend=Aer.get_backend('aer_simulator')
job=backend.run(measure_circ,shots=10)
result=job.result()
counts = job.result().get_counts()
print(counts)

{'0100011100': 1, '0001110100': 1, '0000011100': 1, '1110110100': 3, '0011111000': 4}


In [12]:
max_count = max(counts.items(), key=lambda x: x[1])
print(f'MAX count: {max_count}')

theta = int(max_count[0][::-1], 2) / 2**n_qpe_qbits
print(f'Theta value: {theta}')
print(f'QPE-approximated U-eigenvalue: {np.exp(2*1j*np.pi * theta)}')
print(f'QPE-approximated H-eigenvalue: {-2 * np.pi * theta}')

MAX count: ('0011111000', 4)
Theta value: 0.12109375
QPE-approximated U-eigenvalue: (0.724247082951467+0.6895405447370668j)
QPE-approximated H-eigenvalue: -0.760854470791278


## For different orientation of N2 molecule

In [13]:
driver = PySCFDriver(
    atom="N -1.05068773 0 0; N 1.05068773 0 0",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)


# Run the driver to get the problem
problem = driver.run()

# Define the log output (here using stdout)
flog = sys.stdout

### Apply freezecore transformer
transformer = FreezeCoreTransformer()
problem = transformer.transform(problem)

hamiltonian = problem.hamiltonian.second_q_op()
#print(hamiltonian)
print("\n".join(str(hamiltonian).splitlines()[:10] + ["..."]))

Fermionic Operator
number spin orbitals=16, number terms=3288
  0.24020631525796135 * ( +_0 +_0 -_0 -_0 )
+ -0.02069300789525726 * ( +_0 +_0 -_2 -_0 )
+ 0.2316515617272309 * ( +_0 +_1 -_1 -_0 )
+ 0.021613616445664566 * ( +_0 +_1 -_7 -_0 )
+ -0.02069300789525726 * ( +_0 +_2 -_0 -_0 )
+ 0.2365639590463656 * ( +_0 +_2 -_2 -_0 )
+ 0.23560093073382055 * ( +_0 +_3 -_3 -_0 )
+ 0.23560093073382016 * ( +_0 +_4 -_4 -_0 )
...


In [14]:
print(problem.num_particles)
print(problem.num_spatial_orbitals)
#print(problem.active_orbitals)

(5, 5)
8


In [15]:
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

as_transformer = ActiveSpaceTransformer(2,2) # syntax = AST(num_electrons,num_orbitals)
as_problem = as_transformer.transform(problem)
print(as_problem.num_particles)
print(as_problem.num_spatial_orbitals)

(1, 1)
2


In [16]:
print(as_problem.hamiltonian.electronic_integrals.alpha)

Polynomial Tensor
 "+-":
array([[-7.20070885e-01, -3.24163192e-13],
       [-3.24607281e-13, -8.32704465e-01]])
 "++--":
array([[[[ 4.97554008e-01, -1.21229415e-13],
         [-1.21229415e-13,  4.60134443e-01]],

        [[-1.21229415e-13,  2.04534653e-02],
         [ 2.04534653e-02,  1.18433041e-13]]],


       [[[-1.21229415e-13,  2.04534653e-02],
         [ 2.04534653e-02,  1.18433041e-13]],

        [[ 4.60134443e-01,  1.18433041e-13],
         [ 1.18433041e-13,  5.06340300e-01]]]])


In [24]:
hamiltonian = as_problem.hamiltonian.second_q_op()

#convert second quantized operator into pauli strings using any chemistry encoding of choice. we use jw.
mapper = JordanWignerMapper()

pauli_hamiltonian = mapper.map(hamiltonian)

for pauli, coeff in sorted(pauli_hamiltonian.label_iter()):
    print(f"{coeff.real:+.8f} * {pauli}")

-0.85189406 * IIII
+0.01069309 * IIIZ
+0.06481330 * IIZI
+0.10992024 * IIZZ
+0.01069309 * IZII
+0.12438850 * IZIZ
+0.11503361 * IZZI
+0.00511337 * XXXX
+0.00511337 * XXYY
+0.00511337 * YYXX
+0.00511337 * YYYY
+0.06481330 * ZIII
+0.11503361 * ZIIZ
+0.12658507 * ZIZI
+0.10992024 * ZZII


In [25]:
print(pauli_hamiltonian)

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IIZZ', 'IZII', 'IZIZ', 'ZIII', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.85189406+0.j,  0.01069309+0.j,  0.0648133 +0.j,  0.10992024+0.j,
  0.01069309+0.j,  0.1243885 +0.j,  0.0648133 +0.j,  0.11503361+0.j,
  0.00511337+0.j,  0.00511337+0.j,  0.00511337+0.j,  0.00511337+0.j,
  0.11503361+0.j,  0.12658507+0.j,  0.10992024+0.j])


In [26]:
eig = np.linalg.eigvals(pauli_hamiltonian)
print('NUMPY eig',np.min(eig))

estimator = Estimator()
optimizer = SLSQP()
ansatz = TwoLocal(rotation_blocks=['ry', 'rz'], entanglement_blocks='cz')

vqe = VQE(estimator, ansatz, optimizer)
result = vqe.compute_minimum_eigenvalue(operator=pauli_hamiltonian)
print('VQE eig', result.eigenvalue)

NUMPY eig (-1.1609841579085824+0j)
VQE eig -1.160889109008661


In [27]:
n_qpe_qbits = 10

U = HamiltonianGate(pauli_hamiltonian, 1, label='H')

# Obtain a solution via QPE
total_qbits = U.num_qubits + n_qpe_qbits
measure_circ = QuantumCircuit(total_qbits, n_qpe_qbits)
measure_circ.h([-1, -2])

qpe = PhaseEstimation(n_qpe_qbits, U)
Qpe=qpe.decompose().decompose().decompose().decompose().decompose().decompose().decompose()
measure_circ = measure_circ.compose(Qpe)
#measure_circ.draw("mpl")

measure_circ.measure(range(n_qpe_qbits), range(n_qpe_qbits))

In [28]:
from qiskit_aer import Aer
backend=Aer.get_backend('aer_simulator')
job=backend.run(measure_circ,shots=10)
result=job.result()
counts = job.result().get_counts()
print(counts)

{'1110000100': 1, '1001110100': 1, '1010110100': 2, '0110111000': 1, '0001000100': 2, '1010111000': 1, '0000000000': 2}


In [29]:
max_count = max(counts.items(), key=lambda x: x[1])
print(f'MAX count: {max_count}')

theta = int(max_count[0][::-1], 2) / 2**n_qpe_qbits
print(f'Theta value: {theta}')
print(f'QPE-approximated U-eigenvalue: {np.exp(2*1j*np.pi * theta)}')
print(f'QPE-approximated H-eigenvalue: {-2 * np.pi * theta}')

MAX count: ('1010110100', 2)
Theta value: 0.1767578125
QPE-approximated U-eigenvalue: (0.44412214457042926+0.8959662497561851j)
QPE-approximated H-eigenvalue: -1.1106020904292042


In [30]:
from qiskit_algorithms.phase_estimators.phase_estimator import PhaseEstimator

In [31]:
eig = np.sort(eig)
eig

array([-1.16098416+0.j, -1.11309437+0.j, -1.11309437+0.j, -1.11309437+0.j,
       -1.07218744+0.j, -0.97932409+0.j, -0.97932409+0.j, -0.94067223+0.j,
       -0.87547681+0.j, -0.87547681+0.j, -0.83270447+0.j, -0.83270447+0.j,
       -0.72007089+0.j, -0.72007089+0.j, -0.30202555+0.j,  0.        +0.j])

### Here is the basic layout of the QPE circuit

![quantum phase estimation circuit](qpe_circuit.png)
! <p style="text-align: center;">QPE circuit</p>

n_qpe_qbits = 10

U = HamiltonianGate(pauli_hamiltonian, 1, label='H')

total_qbits = U.num_qubits + n_qpe_qbits
measure_circ = QuantumCircuit(total_qbits, n_qpe_qbits)
measure_circ.h([-1, -2])

qpe = PhaseEstimation(n_qpe_qbits, U)


backend=Aer.get_backend('aer_simulator')
job=backend.run(measure_circ,shots=10)

iqpe = IterativePhaseEstimation(num_iterations = 3, quantum_instance = job)

### **refer to this [link to notebook for iqpe](https://github.com/qiskit-community/ibm-quantum-challenge-spring-2023/blob/main/content/lab_3/lab3.ipynb)**

okay, using AST we have an active space of 4.

In [32]:
pauli_hamiltonian

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IIZZ', 'IZII', 'IZIZ', 'ZIII', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.85189406+0.j,  0.01069309+0.j,  0.0648133 +0.j,  0.10992024+0.j,
  0.01069309+0.j,  0.1243885 +0.j,  0.0648133 +0.j,  0.11503361+0.j,
  0.00511337+0.j,  0.00511337+0.j,  0.00511337+0.j,  0.00511337+0.j,
  0.11503361+0.j,  0.12658507+0.j,  0.10992024+0.j])

This is our hamiltonian